In [2]:
!pip3 install snowflake-connector-python

In [3]:
import pandas as pd

In [4]:
import snowflake.connector

# Set your connection parameters
user='************'
password='************'
account='************'
warehouse='************'
database='************'
schema='************'

try:
    # Establish a connection
    con = snowflake.connector.connect(
        user=user,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
except Exception as e:
    print("error: ", e)


# Create a cursor
cursor = con.cursor()

# -- query to get top 10 trains covering the maximum distance 
cursor.execute("""select train_no, distance
from train_station
where (train_no, SEQ) in (
select train_no, FLOOR(max(SEQ), 0)
from train_station
group by train_no)
order by distance desc limit 10;""")

# Fetch the results
results = cursor.fetchall()

# check the results
for row in results:
    print(row)

('15905', 4260)
('15906', 4256)
('2515', 3939)
('12507', 3932)
('12515', 3932)
('12508', 3930)
('12516', 3930)
('16317', 3769)
('16318', 3765)
('16687', 3663)


In [5]:
# -- query to get top 10 trains covering the maximum distance 
cursor.execute("""select train_no, distance
from train_station
where (train_no, SEQ) in (
select train_no, max(SEQ)
from train_station
group by train_no)
order by distance desc limit 10;""")

# Fetch the results
results = cursor.fetchall()

# Convert the results to a Pandas DataFrame
columns = ["train_no", "distance"]
df = pd.DataFrame(results, columns=columns)
print(df)

  train_no  distance
0    15905      4260
1    15906      4256
2     2515      3939
3    12507      3932
4    12515      3932
5    12508      3930
6    12516      3930
7    16317      3769
8    16318      3765
9    16687      3663


In [6]:
# -- query to get top 10 trains covering the minimum distance 
cursor.execute("""select train_no, distance
from train_station
where (train_no, SEQ) in (
select train_no, max(SEQ)
from train_station
group by train_no)
order by distance asc limit 10;""")

# Fetch the results
results = cursor.fetchall()

# Convert the results to a Pandas DataFrame
columns = ["train_no", "distance"]
df = pd.DataFrame(results, columns=columns)
print(df)

  train_no  distance
0    79488         1
1    79486         1
2    79487         1
3    79489         1
4    79490         1
5    79485         1
6    96015         2
7    96016         2
8    96002         2
9    96009         2


In [7]:
# -- count of unique station codes by states
cursor.execute("""select state, count(DISTINCT(station_code)) 
from stations_info 
where state != '#N/A' and state != '0'
group by state 
order by count(DISTINCT(station_code)) DESC;""")

# Fetch the results
results = cursor.fetchall()

# Convert the results to a Pandas DataFrame
columns = ["state", "count_of_stations"]
df = pd.DataFrame(results, columns=columns)
print(df)

                   state  count_of_stations
0          Uttar Pradesh                309
1            Maharashtra                268
2             Tamil Nadu                203
3                  Assam                175
4              Rajasthan                174
5                Gujarat                164
6         Andhra Pradesh                158
7            West Bengal                153
8         Madhya Pradesh                124
9                  Bihar                113
10             Karnataka                111
11                Kerala                 84
12               Haryana                 83
13                Punjab                 77
14                Odisha                 62
15             Jharkhand                 58
16             Telangana                 55
17          Chhattisgarh                 34
18     Jammu and Kashmir                 29
19                 Delhi                 26
20                   Goa                 20
21           Uttarakhand        

In [8]:
# -- finding busiest stations
cursor.execute("""select station_code, count(station_code)
from train_station
group by station_code
order by count(station_code) desc limit 10;""")

# Fetch the results
results = cursor.fetchall()

# Convert the results to a Pandas DataFrame
columns = ["station_code", "busiest_station"]
df = pd.DataFrame(results, columns=columns)
print(df)

  station_code  busiest_station
0         CSMT             1027
1          KYN              828
2          TNA              796
3         SDAH              745
4          MSB              738
5          HWH              699
6           DR              567
7          DDJ              463
8          CLA              462
9          TBM              434


In [9]:
# -- count of distinct trains running in each state
cursor.execute("""select s.state, count(distinct(st.train_no))
from stations_info s
join train_station st on s.station_code = st.station_code
where s.state != '#N/A' and s.state != '0'
group by s.state
order by count(distinct(st.train_no)) desc;""")

# Fetch the results
results = cursor.fetchall()

# Convert the results to a Pandas DataFrame
columns = ["state", "count_of_trains_running"]
df = pd.DataFrame(results, columns=columns)
print(df)

                   state  count_of_trains_running
0            Maharashtra                     2696
1            West Bengal                     2433
2          Uttar Pradesh                     1853
3             Tamil Nadu                     1803
4         Andhra Pradesh                     1154
5                  Bihar                     1016
6         Madhya Pradesh                      858
7              Rajasthan                      699
8                  Delhi                      698
9                Haryana                      692
10               Gujarat                      686
11             Karnataka                      675
12             Jharkhand                      671
13             Telangana                      628
14                Punjab                      481
15                Odisha                      474
16             Delhi NCT                      355
17                Kerala                      350
18          Chhattisgarh                      278


In [10]:
# -- count of distinct trains that come to any particular stations
cursor.execute("""select station_code, count(distinct(train_no))
from train_station
group by station_code
order by count(distinct(train_no)) desc limit 10;""")

# Fetch the results
results = cursor.fetchall()

# Convert the results to a Pandas DataFrame
columns = ["station_code", "count_of_trains_running"]
df = pd.DataFrame(results, columns=columns)
print(df)

  station_code  count_of_trains_running
0         CSMT                     1027
1          KYN                      828
2          TNA                      796
3         SDAH                      745
4          MSB                      738
5          HWH                      699
6           DR                      567
7          DDJ                      463
8          CLA                      462
9          TBM                      434


In [11]:
# -- get trains route by station
cursor.execute("""select ts.train_no, ts.SEQ, ts.station_code, s.station_name, ts.arrival_time, ts.departure_time
from train_station ts
JOIN train_info t on t.train_no = ts.train_no
JOIN stations_info s on s.station_code = ts.station_code
where ts.train_no = '19571'
order by ts.SEQ;""")

# Fetch the results
results = cursor.fetchall()

# Convert the results to a Pandas DataFrame
columns = ["train_no", "sequence", "station_code", "station_name", "arr_time", "depart_time"]
df = pd.DataFrame(results, columns=columns)
df.set_index("train_no", inplace=True)
print(df)

          sequence station_code  station_name  arr_time depart_time
train_no                                                           
19571            1          RJT        RAJKOT  07:15:00    07:15:00
19571            2         BKNG   BHAKTINAGAR  07:25:00    07:26:00
19571            3          GDL        GONDAL  08:04:00    08:05:00
19571            4          VRR        VIRPUR  08:35:00    08:36:00
19571            5          NUD      NAVAGADH  08:48:00    08:49:00
19571            6          JLR   JETALSAR JN  09:00:00    09:15:00
19571            7          DJI       DHORAJI  09:34:00    09:35:00
19571            8           UA        UPLETA  10:05:00    10:06:00
19571            9          BHY    BHAYAVADAR  10:17:00    10:18:00
19571           10          PLM   PANELI MOTI  10:27:00    10:28:00
19571           11          JDH   JAM JODHPUR  10:41:00    10:42:00
19571           12          WAB         BALWA  10:49:00    10:50:00
19571           13          WSJ  WANSJALIYA J  1